In [20]:
# 기사 라벨링 된 것 파일 불러오기
import pandas as pd
df_article = pd.read_csv('ngramized2_sentence_naver_interest_article.csv')
df_article = df_article[['date', 'Labelling','n_gram']]

In [25]:
# 채권분석 라벨링된 파일 불러오기
df_bond = pd.read_csv('ngramized2_bond.csv')
df_bond = df_bond[['date','Labelling', 'n_gram']]

In [38]:
df_trainset = pd.concat([df_article, df_bond])

In [39]:
# 기사 & 채권분석 concat -> [] 빼주기, text가 없는 날짜 빼주기
df_trainset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2971180 entries, 0 to 3077
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   date       object
 1   Labelling  object
 2   n_gram     object
dtypes: object(3)
memory usage: 90.7+ MB


In [41]:
df_trainset = df_trainset[df_trainset['n_gram'] != '[]'] #엔그램 없는 것 빼기

In [42]:
from tqdm.notebook import tqdm

In [43]:
from collections import defaultdict
n_gram = defaultdict(lambda:[0,0,0]) # total_count, up_count, down_count

for i in tqdm(range(len(df_trainset))):
    texts = df_trainset['n_gram'].iloc[i].replace("[","").replace("]","").replace("'","").replace(",","")
    texts = texts.split()
    for text in texts:
        n_gram[text][0] += 1
        if df_trainset['Labelling'].iloc[i] == 'up':
            n_gram[text][1] += 1
        elif df_trainset['Labelling'].iloc[i] == 'down':
            n_gram[text][2] += 1

In [44]:
ngram, total_count, up_count, down_count = [],[],[],[]
for i, doc in n_gram.items():
    ngram.append(i)
    total_count.append(doc[0])
    up_count.append(doc[1])
    down_count.append(doc[2])

In [45]:
df_polar_score = pd.DataFrame([ngram, total_count, up_count, down_count],index=['ngram','total_count','up_count','down_count'])
df_polar_score = df_polar_score.T

In [46]:
# 총 개수가 15이하인 것 delete
df_polar_score = df_polar_score[df_polar_score['total_count']>=15]

In [52]:
df_polar_score.info()
len(df_polar_score[df_polar_score['down_count']==0])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45637 entries, 0 to 137077
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ngram           45637 non-null  object 
 1   total_count     45637 non-null  object 
 2   up_count        45637 non-null  object 
 3   down_count      45637 non-null  object 
 4   bayes_up        45637 non-null  float64
 5   bayes_down      45637 non-null  float64
 6   polarity_score  45637 non-null  float64
 7   polarity        45637 non-null  object 
dtypes: float64(3), object(5)
memory usage: 3.1+ MB


179

In [48]:
# 스팸 필터링 만들기
import numpy as np

df_polar_score['bayes_up'] = np.zeros(len(df_polar_score))
df_polar_score['bayes_down'] = np.zeros(len(df_polar_score))

total_up = df_polar_score['up_count'].sum()
total_down = df_polar_score['down_count'].sum()

for i in range(len(df_polar_score)):
    df_polar_score['bayes_up'].iloc[i] = (0.5 + df_polar_score['up_count'].iloc[i])/ (2 *0.5 + total_up)
    df_polar_score['bayes_down'].iloc[i] = (0.5 + df_polar_score['down_count'].iloc[i]) / (2 *0.5 + total_down)

C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
df_polar_score['polarity_score'] = df_polar_score['bayes_up']/df_polar_score['bayes_down']
df_polar_score['polarity'] = np.zeros(len(df_polar_score))

for i in range(len(df_polar_score)):
    if df_polar_score['polarity_score'].iloc[i] > 1.3:
        df_polar_score['polarity'].iloc[i] = 'Hawkish'
    elif df_polar_score['polarity_score'].iloc[i] < (1/1.3):
        df_polar_score['polarity'].iloc[i] = 'Dovish' 
    else:
        df_polar_score['polarity'].iloc[i] = 'Neutral'
df_polar_score[:10]

,ngram,total_count,up_count,down_count,bayes_up,bayes_down,polarity_score,polarity
0,은행/NNG,125992,51104,51371,0.003075,0.003205,0.959290,Neutral
1,수수료/NNG,13206,5177,5152,0.000311,0.000321,0.968981,Neutral
2,수입/NNG,14707,6114,5617,0.000368,0.000350,1.049617,Neutral
3,급급/NNG,260,89,113,0.000005,0.000007,0.760397,Dovish
4,일단/MAG,9882,4135,3910,0.000249,0.000244,1.019785,Neutral
5,팔/VV,12705,4800,5188,0.000289,0.000324,0.892191,Neutral
6,상품/NNG,46644,18910,18635,0.001138,0.001163,0.978532,Neutral
7,판매/NNG,32966,14123,12340,0.000850,0.000770,1.103628,Neutral
8,경쟁/NNG,12107,5183,4737,0.000312,0.000296,1.055084,Neutral
9,나서/VV,46222,17748,18800,0.001068,0.001173,0.910343,Neutral


In [50]:
df_hawkish = df_polar_score[df_polar_score['polarity']=='Hawkish']
df_dovish = df_polar_score[df_polar_score['polarity']=='Dovish']
len(df_hawkish), len(df_dovish)

(11442, 13311)

In [51]:
df_hawkish.to_csv('version2_df_hawkish_dict.csv',index=False)
df_dovish.to_csv('version2_df_dovish_dict.csv',index=False)